In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import IPython.display as ipd
import numpy as np, pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.file_utils import ModelOutput
from transformers import AutoConfig, Wav2Vec2Processor
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

from pathlib import Path
from tqdm import tqdm
import torchaudio, os, sys, json, pickle, librosa

from dataclasses import dataclass
from typing import Optional, Tuple
from tqdm import tqdm

In [3]:
# @dataclass
# class SpeechClassifierOutput(ModelOutput):
# #     loss: Optional[torch.FloatTensor] = None
# #     logits: torch.FloatTensor = None
#     hidden_states: Optional[Tuple[torch.FloatTensor]]
# #     attentions: Optional[Tuple[torch.FloatTensor]]
# #     h1: Optional[Tuple[torch.FloatTensor]] = None
#     h2: Optional[Tuple[torch.FloatTensor]] = None

# class Wav2Vec2ClassificationHead(nn.Module):
#     """Head for wav2vec classification task."""

#     def __init__(self, config):
#         super().__init__()
#         self.dense1 = nn.Linear(config.hidden_size, 300)
#         self.dense2 = nn.Linear(300, 100)
#         self.dropout = nn.Dropout(config.final_dropout)
#         self.out_proj = nn.Linear(100, config.num_labels)


#     def forward(self, features, **kwargs):
#         x = features
#         x = self.dropout(x)
#         x = self.dense1(x)
#         x1 = torch.tanh(x)
#         x2 = self.dropout(x1)
#         x2 = self.dense2(x2)
#         x2 = torch.tanh(x2)
#         x3 = self.dropout(x2)        
#         x3 = self.out_proj(x3)
#         return x1, x2, x3

class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        config.pooling_mode = "mean"
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
#         self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()
        
    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()
        for module in self.wav2vec2.encoder.layers[:10]:
            for param in module.parameters():
                param.requires_grad = False

    def merged_strategy(self, hidden_states, mode="mean"):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception("The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
#         print("type of hidden_states is = ", type(hidden_states))
        return hidden_states
#         hidden_rep, logits = self.classifier(hidden_states)
#         h1, h2, logits = self.classifier(hidden_states)

#         loss = None
#         if labels is not None:
#             if self.config.problem_type is None:
#                 if self.num_labels == 1:
#                     self.config.problem_type = "regression"
#                 elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
#                     self.config.problem_type = "single_label_classification"
#                 else:
#                     self.config.problem_type = "multi_label_classification"

#             if self.config.problem_type == "regression":
#                 loss_fct = MSELoss()
#                 loss = loss_fct(logits.view(-1, self.num_labels), labels)
#             elif self.config.problem_type == "single_label_classification":
#                 loss_fct = CrossEntropyLoss()
#                 loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
#             elif self.config.problem_type == "multi_label_classification":
#                 loss_fct = BCEWithLogitsLoss()
#                 loss = loss_fct(logits, labels)

#         if not return_dict:
#             output = (h1 + h2 + logits,) + outputs[2:]
#             return ((loss,) + output) if loss is not None else output
        
#         return SpeechClassifierOutput(
# #             loss=loss,
# #             logits=logits,
#             hidden_states=outputs.hidden_states,
# #             attentions=outputs.attentions,
# #             h1=h1,
# #             h2=h2
#         )

In [4]:
# !export CUDA_VISIBLE_DEVICES=0
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# torch.cuda.current_device()

In [6]:
!gpustat

swara  Tue May 24 18:02:56 2022
[0] GeForce GTX 1080 Ti | 36'C,   0 % |  8725 / 11178 MB | mayank(8721M)
[1] GeForce GTX 1080 Ti | 35'C,   0 % |  2117 / 11178 MB | mayank(2113M)
[2] GeForce GTX 1080 Ti | 34'C,   0 % |     1 / 11178 MB |
[3] GeForce GTX 1080 Ti | 45'C,   0 % |  6705 / 11177 MB | mayank(6701M)


In [7]:
import os
# print(os.environ['CUDA_VISIBLE_DEVICES'])
os.environ['CUDA_VISIBLE_DEVICES']='2'
print(os.environ['CUDA_VISIBLE_DEVICES'])
device='cuda'

2


In [8]:
# model_name_or_path = "/mnt/data/aman/mayank/MTP/mount_points/jan_19/Error-Driven-ASR-Personalization/MCV_accent/data/dristi_accent-recognition/checkpoint-6400/"
model_name_or_path = "/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/w2v2/w2v2_timit/checkpoint-7000/"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
sampling_rate = processor.feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at /home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/w2v2/w2v2_timit/checkpoint-7000/ were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
!gpustat

swara  Tue May 24 18:03:06 2022
[0] GeForce GTX 1080 Ti | 36'C,   0 % |  8725 / 11178 MB | mayank(8721M)
[1] GeForce GTX 1080 Ti | 36'C,   0 % |  2117 / 11178 MB | mayank(2113M)
[2] GeForce GTX 1080 Ti | 36'C,   0 % |  1201 / 11178 MB | mayank(1197M)
[3] GeForce GTX 1080 Ti | 45'C,   0 % |  6705 / 11177 MB | mayank(6701M)


In [10]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def predict(path, sampling_rate):
#     print(path)
    speech = speech_file_to_array_fn(path, sampling_rate)
    features = processor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    input_values = features.input_values.to(device)
    attention_mask = None

    with torch.no_grad():
        op = model(input_values, attention_mask=attention_mask)
#         logits = op.logits
#         h1 = op.h1
#         h2 = op.h2
#         print(list(op))
        pooled_w2v2_features = op
        
#     scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
#     outputs = [{"Accent": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
#     return outputs, h1, h2, logits
    return pooled_w2v2_features

def prediction(df_row):
    if 'path' in df_row: path = df_row["path"]
    else: path = df_row["audio_filepath"]
    speech, sr = torchaudio.load(path)
    speech = speech[0].numpy().squeeze()
    speech = librosa.resample(np.asarray(speech), sr, sampling_rate)
#     outputs, h1, h2, h3 = predict(path, sampling_rate)
    outputs = predict(path, sampling_rate)
    return outputs

def extract_features(file_list, file_dir):
    with open(file_dir.replace('.json', '_w2v2.file'), 'wb') as f:
        for file in tqdm(file_list, position=0, leave=True):
            w2v2_features = prediction(file).cpu().detach().numpy()
            pickle.dump(w2v2_features, f)

In [12]:
base_dir = './'

# accents = [ 'malayalam_male_english', 'kannada_male_english', 'manipuri_female_english', 'rajasthani_male_english',
#         'hindi_male_english', 'gujarati_female_english', 'assamese_female_english', 'tamil_male_english']

accents = [ 'kannada_male_english', 'manipuri_female_english', 'rajasthani_male_english',
        'hindi_male_english', 'gujarati_female_english', 'assamese_female_english', 'tamil_male_english']


for accent in tqdm(accents):
    manifests_path = base_dir + f'/{accent}/'
    !mkdir -p $manifests_path/w2v2/
#     manifests_path = base_dir + accent + '/manifests/'
    print('_'*20)
    print(accent)

    seed_file_dir = manifests_path + 'w2v2/seed.json'
    seed_file_path = manifests_path + 'seed.json'
    seed_file = open(seed_file_path)
    seed_list = [json.loads(line.strip()) for line in seed_file]

    print('seed_file_starting')
    print(seed_file_dir)
    extract_features(seed_list, seed_file_dir)
    print(len(seed_list))
    print('seed_file_ending ...\n')
    
    
    selection_file_dir = manifests_path + 'w2v2/selection.json'
    selection_file_path = manifests_path + 'selection.json'
    selection_file = open(selection_file_path)
    selection_list = [json.loads(line.strip()) for line in selection_file]
    
    print('selection_file_starting')
    extract_features(selection_list, selection_file_dir)
    print(len(selection_list))
    print('selection_file_ending ...\n\n')
    
    
    test_file_dir = manifests_path + 'w2v2/test.json'
    test_file_path = manifests_path + 'test.json'
    test_file = open(test_file_path)
    test_list = [json.loads(line.strip()) for line in test_file]

    print('test_file_starting')
    extract_features(test_list, test_file_dir)
    print(len(test_list))
    print('test_file_ending ...\n\n')
    
    
#     dev_file_dir = manifests_path + 'w2v2/'
#     dev_file_path = manifests_path + 'dev.json'
#     dev_file = open(dev_file_path)
#     dev_list = [json.loads(line.strip()) for line in dev_file]

#     print('dev_file_starting')
#     print(dev_file_dir)
#     extract_features(dev_list, dev_file_dir)
#     print(len(dev_list))
#     print('dev_file_ending ...\n')

  0%|          | 0/7 [00:00<?, ?it/s]

____________________
kannada_male_english
seed_file_starting
.//kannada_male_english/w2v2/seed.json


100%|██████████| 50/50 [00:11<00:00,  4.23it/s]


50
seed_file_ending ...

selection_file_starting


100%|██████████| 3244/3244 [15:56<00:00,  3.39it/s]


...

1401
test_file_ending ...


____________________
assamese_female_english
seed_file_starting
.//assamese_female_english/w2v2/seed.json


100%|██████████| 50/50 [00:15<00:00,  3.29it/s]


50
seed_file_ending ...

selection_file_starting


 74%|███████▎  | 4248/5765 [20:49<05:45,  4.39it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 7/7 [3:27:41<00:00, 1780.15s/it]

2263
test_file_ending ...




In [ ]:
for b1 in [2000, 3500]:
    for target in [20]:
        for fxn in ["FL2MI"]:
            for ft in ["39"]:
                for accent in [ 'malayalam_male_english', 'kannada_male_english', 'manipuri_female_english', 'rajasthani_male_english', 
                               'hindi_male_english', 'gujarati_female_english', 'assamese_female_english', 'tamil_male_english']:


                    base_dir = f'./{accent}/all/budget_{b1}/target_{target}/{fxn}/{ft}'
                    manifests_path = base_dir + f'/train.json'
                    print('_'*20)
                    print(f'Setting: {base_dir}')
                    train_file_path = manifests_path
                    train_file_dir = train_file_path
                    train_file = open(train_file_path)
                    train_list = [json.loads(line.strip()) for line in train_file] 

                    print('train_file_starting')
                    print(train_file_dir)
                    extract_features(train_list, train_file_dir)
                    print(len(train_list))
                    print('train_file_ending ...\n')

____________________
Setting: ./malayalam_male_english/all/budget_2000/target_20/FL2MI/39
train_file_starting
./malayalam_male_english/all/budget_2000/target_20/FL2MI/39/train.json


100%|██████████| 1682/1682 [07:31<00:00,  3.73it/s]


1682
train_file_ending ...

____________________
Setting: ./kannada_male_english/all/budget_2000/target_20/FL2MI/39
train_file_starting
./kannada_male_english/all/budget_2000/target_20/FL2MI/39/train.json


100%|██████████| 1672/1672 [07:25<00:00,  3.75it/s]


1672
train_file_ending ...

____________________
Setting: ./manipuri_female_english/all/budget_2000/target_20/FL2MI/39
train_file_starting
./manipuri_female_english/all/budget_2000/target_20/FL2MI/39/train.json


...

100%|██████████| 3202/3202 [13:14<00:00,  4.03it/s]


3202
train_file_ending ...

____________________
Setting: ./hindi_male_english/all/budget_3500/target_20/FL2MI/39
train_file_starting
./hindi_male_english/all/budget_3500/target_20/FL2MI/39/train.json


100%|██████████| 3365/3365 [15:24<00:00,  3.64it/s]


3365
train_file_ending ...

____________________
Setting: ./gujarati_female_english/all/budget_3500/target_20/FL2MI/39
train_file_starting
./gujarati_female_english/all/budget_3500/target_20/FL2MI/39/train.json


 73%|███████▎  | 1647/2269 [10:52<05:06,  2.03it/s]

In [12]:
# !gpustat

swara  Tue May 24 20:54:00 2022
[0] GeForce GTX 1080 Ti | 50'C,   0 % |  8725 / 11178 MB | mayank(8721M)
[1] GeForce GTX 1080 Ti | 86'C, 100 % |  7973 / 11178 MB | mayank(7969M)
[2] GeForce GTX 1080 Ti | 64'C,   0 % |  3411 / 11178 MB | mayank(3407M)
[3] GeForce GTX 1080 Ti | 50'C,   0 % |  6705 / 11177 MB | mayank(6701M)
